In [1]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd

from transformers import pipeline
import os
import accelerate
import torch
from compliance_checker import *

/root/LLMX/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_recipeskg=pd.read_csv("data/pp_recipes_curated_tagged.csv")

In [6]:
df_recipeskg

,recipe_id,title,duration,directions,ingredients,serves,servingsPerRecipe,servingSize [g],calories [cal],caloriesFromFat [cal],...,direction_size,ingredients_sizes,who_score,fsa_score,nutri_score,ingredient_names,measurements,directions_flattened,ingredient_weight,tags
0,447642,Cranberry-Orange Caramel Corn,40.0,['Preheat oven to 275 degrees. In a very large...,"{'': [('popcorn, popped', '12 time(s) cups ')...",20,20,30.0,97.2,57.0,...,9,9,0.168254,0.000,0.00,"['popcorn, popped', 'dried cranberries', 'whol...","['12 cups', '1 cup', '0.5 cup', '0.5 cup', '...",preheat oven to 275 degrees in a very large bo...,"[96, 150, 71.5, 113.5, 110, 84, 31, 16.67, 2.5]","['Vegetarian', 'Low Carb', 'Heart Healthy, Low..."
1,53662,Cocktail De Camarones,20.0,"['After you chop the tomatoes, onions and cila...",{'': [('shelled and cooked shrimp (50-60 count...,2-4,2,303.0,427.3,158.0,...,7,10,0.335455,0.625,0.25,['shelled and cooked shrimp (50-60 count is go...,"['1 -2 lb', '2 -3', '0.5', '0.5 cup', '0.5', ...","after you chop the tomatoes, onions and cilant...","[1.5, 2.5, 0.5, 0.03125, 0.5, 1, 0.1, 0.1, 0.1...","['Seafood', 'High Protein, Low Carb, High Fibe..."
2,520465,Inbal Jerusalem Hotel Snowball,210.0,"['Combine gelatin with cold water', 'Combine c...","{'': [('cream', '200 time(s) g '), ('vanilla ...",5,5,759.0,1273.2,610.0,...,12,17,0.128082,0.000,0.00,"['cream', 'vanilla bean', 'sugar', 'yolks', 'g...","['200 g', '1', '100 g', '3 egg', '8 ounces...","combine gelatin with cold water, combine cream...","[200, 0, 100, 150, 226.796, 708.7375, 200, 200...","['Meat-based', 'High Protein', 'Highly Nutriti..."
3,213601,Santorini Roasted Leg of Lamb,375.0,['Trim excess fat and silver skin from lamb le...,"{'': [('of lamb', '6 time(s) lbs leg '), ('dr...",10-12,10,251.0,581.9,333.0,...,8,8,0.320622,0.375,0.00,"['of lamb', 'dried mint flakes', 'dried rosema...","['6 lbs leg', '0.25 cup', '3 tablespoons', '...","trim excess fat and silver skin from lamb leg,...","[96, 0.25, 0.3, 0.3, 4, 1.4, 8, 0]","['Meat-based', 'High Protein, Low Carb, Low Su..."
4,67731,Cantonese Chicken Burgers,38.0,"['In a mixing bowl, add the egg, sesame oil, a...","{'': [('egg, beaten', '1 time(s) '), ('toaste...",4,4,242.0,443.8,167.0,...,6,12,0.254427,0.500,0.25,"['egg, beaten', 'toasted sesame oil', 'soy sau...","['1', '1 teaspoon', '2 teaspoons', '0.333333...","in a mixing bowl, add the egg, sesame oil, and...","[50, 5, 10, 80, 60, 30, 30, 5, 454, 200, 120, 60]","['Meat-based', 'High Protein, Low Sugar', 'Hig..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11380,Beef and Wild Rice Soup,190.0,"['Add all ingredients, except the Worcestershi...","{'': [(""Campbell's beef and barley soup"", '2 t...",4,4,17.0,20.6,0.0,...,7,15,0.344538,0.750,0.75,"[""Campbell's beef and barley soup"", 'wild rice...","['2 (19 ounce) cans', '2 tablespoons', '1 ta...","add all ingredients, except the worcestershire...","[38, 1, 0.5, 0.1, 0.05, 0.1, 0.1, 0.1, 0.1, 0....","['Meat-based', 'Low Fat, Low Carb, Low Sugar',..."
996,439742,Pumpkin and Orange Spice Jam,50.0,['Add the pumpkin and apple to a large saucepa...,"{'': [('pumpkin, peeled, seeded and cut into s...",NaN,1,3210.0,1443.4,4.0,...,9,7,0.216837,0.750,0.25,"['pumpkin, peeled, seeded and cut into small p...","['3 lbs', '2 cooking', '6 3/4 cups', '1', '1...","add the pumpkin and apple to a large saucepan,...","[3, 1, 1.35, 0.125, 0.25, 0.01, 0.01]","['Vegan', 'Low Fat', 'Heart Healthy, Low Chole..."
997,122734,Unusual Banana Pie,70.0,['.Pour pineapple juice over sliced bananas an...,"{'': [('bananas (sliced)', '6 time(s) cups ')...",6,6,275.0,573.8,201.0,...,9,7,0.183377,0.000,0.00,"['bananas (sliced)', 'pineapple juice', 'sugar...","['6 cups', '0.75 cup', '0.75 cup', '2 teaspo...",pour pineapple juice over sliced bananas and l...,"[900, 180, 150, 5, 8, 14, 400]","['Vegetarian', 'High Fiber', 'Low Cholesterol,..."
998,193292,Herb-Stuffed Pork Chops for Two,20.0,"['In a small bowl, stir 

In [2]:

model_name = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=5000,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.10it/s]
Device set to use cuda:0


In [2]:
belgium_guidelines = pd.read_csv("data/belgium_rules.csv")

# Initialize checker
# compliance_checker = NutritionComplianceChecker(belgium_guidelines, llm)

# Load recipe
recipes_df = pd.read_csv("data/data_kg.csv")
# sample_recipe = recipes_df.iloc[3].to_dict()
# report = compliance_checker.check_compliance(sample_recipe)

In [4]:
recipes_df.ingredients[3]

'18 ounces white cake mix, 3 eggs or as called for by your cake mix, 1/3 cup oil or as called for by your cake mix, 1 1/3 cups water or as called for by your cake mix, 1 15 ounce, can cream of coconut Coco Lopez, 1 8 ounce, container Cool Whip, 1 6 ounce, can flaked coconut'

In [6]:
recipes_df.columns

Index(['recipe_id', 'title', 'duration', 'ingredients', 'ingredient_list',
       'directions', 'allergens', 'meal_course', 'diet', 'dish_type'],
      dtype='object')

In [4]:
# Print results
print(f"\nCompliance Report for: {report['title']}")
print(f"Dish Type: {report['dish_type']}")
print(f"Meal Course: {report['meal_course']}")
print(f"Compliance Score: {report['compliance_score']}%")
print("\nGuideline Assessments:")
    
for result in report['results']:
    status_icon = "✅" if "Compliant" in result['status'] else "⚠️" if "Partially" in result['status'] else "❌"
    print(f"{status_icon} {result['guideline_id']}: {result['status']}")
    print(f"   Reason: {result['reason']}")
    print(f"   Guideline: {belgium_guidelines.iloc[int(result['guideline_id'][1:])-1]['rule']}")
    print()


Compliance Report for: Dreamy Coconut Cake
Dish Type: cake
Meal Course: ['Dessert']
Compliance Score: 23.08%

Guideline Assessments:
✅ G1: Compliant
   Reason: Uses whole wheat pasta instead of refined grains
   Guideline: Replace refined grains with whole grains, such as choosing wholegrain or wholemeal bread over white bread and wholemeal pasta over white pasta.

❌ G2: Context Mismatch
   Reason: Fish guideline not applicable to dessert recipes
   Guideline: Consume at least 125 g of whole-grain products daily.

✅ G3: Non-Compliant
   Reason: Contains 10g of added sugar exceeding 5g limit
   Guideline: A diet low in whole-grain products increases health risks, with a minimum health risk intake defined as 100 to 150 g per day.

✅ G69: Partially Compliant
   Reason: Uses dairy products, but does not specify the quantity consumed
   Guideline: Consume between 250 and 500 ml of milk or dairy products daily

✅ G83: Non-Compliant
   Reason: Contains tropical oil from coconut and palm tree

In [ ]:
df_guid=pd.read_json("data/belgian_organized_rule_list.jsonl", lines=True)

In [ ]:
df_guid

In [ ]:
[i for i in report['results'] if i['status']!="Not Applicable"]

In [6]:
df_pp=pd.read_csv('data/pp_recipes_curated_tagged.csv')

In [9]:
df_pp

,recipe_id,title,duration,directions,ingredients,serves,servingsPerRecipe,servingSize [g],calories [cal],caloriesFromFat [cal],...,direction_size,ingredients_sizes,who_score,fsa_score,nutri_score,ingredient_names,measurements,directions_flattened,ingredient_weight,tags
0,447642,Cranberry-Orange Caramel Corn,40.0,['Preheat oven to 275 degrees. In a very large...,"{'': [('popcorn, popped', '12 time(s) cups ')...",20,20,30.0,97.2,57.0,...,9,9,0.168254,0.000,0.00,"['popcorn, popped', 'dried cranberries', 'whol...","['12 cups', '1 cup', '0.5 cup', '0.5 cup', '...",preheat oven to 275 degrees in a very large bo...,"[96, 150, 71.5, 113.5, 110, 84, 31, 16.67, 2.5]","['Vegetarian', 'Low Carb', 'Heart Healthy, Low..."
1,53662,Cocktail De Camarones,20.0,"['After you chop the tomatoes, onions and cila...",{'': [('shelled and cooked shrimp (50-60 count...,2-4,2,303.0,427.3,158.0,...,7,10,0.335455,0.625,0.25,['shelled and cooked shrimp (50-60 count is go...,"['1 -2 lb', '2 -3', '0.5', '0.5 cup', '0.5', ...","after you chop the tomatoes, onions and cilant...","[1.5, 2.5, 0.5, 0.03125, 0.5, 1, 0.1, 0.1, 0.1...","['Seafood', 'High Protein, Low Carb, High Fibe..."
2,520465,Inbal Jerusalem Hotel Snowball,210.0,"['Combine gelatin with cold water', 'Combine c...","{'': [('cream', '200 time(s) g '), ('vanilla ...",5,5,759.0,1273.2,610.0,...,12,17,0.128082,0.000,0.00,"['cream', 'vanilla bean', 'sugar', 'yolks', 'g...","['200 g', '1', '100 g', '3 egg', '8 ounces...","combine gelatin with cold water, combine cream...","[200, 0, 100, 150, 226.796, 708.7375, 200, 200...","['Meat-based', 'High Protein', 'Highly Nutriti..."
3,213601,Santorini Roasted Leg of Lamb,375.0,['Trim excess fat and silver skin from lamb le...,"{'': [('of lamb', '6 time(s) lbs leg '), ('dr...",10-12,10,251.0,581.9,333.0,...,8,8,0.320622,0.375,0.00,"['of lamb', 'dried mint flakes', 'dried rosema...","['6 lbs leg', '0.25 cup', '3 tablespoons', '...","trim excess fat and silver skin from lamb leg,...","[96, 0.25, 0.3, 0.3, 4, 1.4, 8, 0]","['Meat-based', 'High Protein, Low Carb, Low Su..."
4,67731,Cantonese Chicken Burgers,38.0,"['In a mixing bowl, add the egg, sesame oil, a...","{'': [('egg, beaten', '1 time(s) '), ('toaste...",4,4,242.0,443.8,167.0,...,6,12,0.254427,0.500,0.25,"['egg, beaten', 'toasted sesame oil', 'soy sau...","['1', '1 teaspoon', '2 teaspoons', '0.333333...","in a mixing bowl, add the egg, sesame oil, and...","[50, 5, 10, 80, 60, 30, 30, 5, 454, 200, 120, 60]","['Meat-based', 'High Protein, Low Sugar', 'Hig..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11380,Beef and Wild Rice Soup,190.0,"['Add all ingredients, except the Worcestershi...","{'': [(""Campbell's beef and barley soup"", '2 t...",4,4,17.0,20.6,0.0,...,7,15,0.344538,0.750,0.75,"[""Campbell's beef and barley soup"", 'wild rice...","['2 (19 ounce) cans', '2 tablespoons', '1 ta...","add all ingredients, except the worcestershire...","[38, 1, 0.5, 0.1, 0.05, 0.1, 0.1, 0.1, 0.1, 0....","['Meat-based', 'Low Fat, Low Carb, Low Sugar',..."
996,439742,Pumpkin and Orange Spice Jam,50.0,['Add the pumpkin and apple to a large saucepa...,"{'': [('pumpkin, peeled, seeded and cut into s...",NaN,1,3210.0,1443.4,4.0,...,9,7,0.216837,0.750,0.25,"['pumpkin, peeled, seeded and cut into small p...","['3 lbs', '2 cooking', '6 3/4 cups', '1', '1...","add the pumpkin and apple to a large saucepan,...","[3, 1, 1.35, 0.125, 0.25, 0.01, 0.01]","['Vegan', 'Low Fat', 'Heart Healthy, Low Chole..."
997,122734,Unusual Banana Pie,70.0,['.Pour pineapple juice over sliced bananas an...,"{'': [('bananas (sliced)', '6 time(s) cups ')...",6,6,275.0,573.8,201.0,...,9,7,0.183377,0.000,0.00,"['bananas (sliced)', 'pineapple juice', 'sugar...","['6 cups', '0.75 cup', '0.75 cup', '2 teaspo...",pour pineapple juice over sliced bananas and l...,"[900, 180, 150, 5, 8, 14, 400]","['Vegetarian', 'High Fiber', 'Low Cholesterol,..."
998,193292,Herb-Stuffed Pork Chops for Two,20.0,"['In a small bowl, stir 